# Dataset preparing


In [ ]:
#data downloading

%cd datasets
%cd RDD2022
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Japan.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_India.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Czech.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Norway.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_United_States.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_MotorBike.zip
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_Drone.zip
!unzip RDD2022_Japan.zip
!unzip RDD2022_India.zip
!unzip RDD2022_Czech.zip
!unzip RDD2022_Norway.zip
!unzip RDD2022_United_States.zip
!unzip RDD2022_China_Drone.zip
!unzip RDD2022_China_MotorBike.zip
!mv RDD2022_Japan Japan
!mv RDD2022_India India
!mv RDD2022_Czech Czech 
!mv RDD2022_Norway Norway
!mv RDD2022_United_States United_States
!mv RDD2022_China_Drone China_Drone
!mv RDD2022_China_MotorBike China_MotorBike

# !ls -a

# data pre-processing for yolo

In [6]:
%pip install Pillow
%pip install tqdm
%cd datasets/RDD2022
!python gene_train_val.py
%cd ../..
!python datasets/RDD2022/xml2yolo.py --class_file datasets/RDD2022/damage_classes.txt --input_file datasets/RDD2022/train.txt

total files: 38384
D00 : 13548
D10 : 7709
D40 : 3674
D20 : 8412


# data pre-processing for mmdetection

In [7]:
!python datasets/RDD2022/gene_file_list.py

# test data preparing

In [18]:
!mkdir datasets/RDD2022/test1_images
!cp datasets/RDD2022/China_MotorBike/test/images/* datasets/RDD2022/test1_images/
!cp datasets/RDD2022/Czech/test/images/* datasets/RDD2022/test1_images/
!cp datasets/RDD2022/Inida/test/images/* datasets/RDD2022/test1_images/
!cp datasets/RDD2022/Norway/test/images/* datasets/RDD2022/test1_images/
!cp datasets/RDD2022/Japan/test/images/* datasets/RDD2022/test1_images/
!cp datasets/RDD2022/United_States/test/images/* datasets/RDD2022/test1_images/

mkdir: cannot create directory ‘datasets/RDD2022/test1_images’: File exists
^C
cp: cannot stat ‘datasets/RDD2022/China/test/images/*’: No such file or directory
cp: cannot stat ‘datasets/RDD2022/China/test/images/*’: No such file or directory
cp: cannot stat ‘datasets/RDD2022/China/test/images/*’: No such file or directory
cp: cannot stat ‘datasets/RDD2022/China/test/images/*’: No such file or directory


# YOLOv5 training and inferencing

In [ ]:
%cd yolov5
!ln -s ../datasets datasets 

#training
#pretrained weights
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5x.pt
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5x6.pt
!python train.py --weights yolov5x6.pt --cfg models/hub/yolov5x6.yaml --hyp data/hyps/hyp.scratch-med.yaml --data data/rdd.yaml --batch-size 64 --imgsz 1280 --noval --device 0,1,2,3,4,5,6,7 --sync-bn --save-period 5
!python train.py --weights yolov5x.pt --cfg models/yolov5x.yaml --hyp data/hyps/hyp.scratch-med.yaml --data data/rdd.yaml --batch-size 64 --imgsz 640 --noval --device 0,1,2,3,4,5,6,7 --sync-bn --save-period 5

#inferencing 
! python detect.py --weights {your weight}.pt --source datasets/RDD2022/test1_images/ --imgsz 640 --conf-thres 0.20 --iou-thres 0.9999 --device 0 --nosave --agnostic-nms --filename {your filename}

# YOLOv7 training and inferencing

In [ ]:
%cd ..
%cd yolov7
!ln -s ../datasets datasets

#training
#pretrained weights 
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x.pt
!python train.py --weights yolov7x.pt --cfg cfg/training/yolov7x.yaml --data data/rdd.yaml --batch-size 64 --img-size 640 --notest --device 0,1,2,3,4,5,6,7 --sync-bn --save_period 5 

#inferencing 
!python detect.py --weights {your weights}.pt --source datasets/RDD2022/test1_images/ --img-size 800 --conf-thres 0.01 --iou-thres 0.5 --device 0 --nosave --filename {your filename}

# mmdetection training and inferencing

In [16]:
%cd ..
%cd mmdetection
!ln -s ../datasets datasets

#training 
#pretrained weights 
!mkdir checkpoints
% cd checkpoints 
!wget https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22k.pth
!wget https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth
% cd..

#Faster_Swin_l_w7_ms_1and2
!tools/dist_train.sh configs/swin/faster_swin_l.py 8

#Faster_Swin_l_w12_DeformRoI_ms1and2
!tools/dist_train.sh configs/swin/faster_swin_l12_deform_1.py 4
#Faster_Swin_l_w12_DeformRoI_ms1and2
!tools/dist_train.sh configs/swin/faster_swin_l12_deform_3.py 4


#inferencing
!python inference.py configs/swin/faster_swin_l.py Faster_Swin_l_w7_ms_1_2.pth datasets/RDD2022/test1_images/ 640 0.5 0.9999 test.txt
!python inference.py {configs} {checkpoints} datasets/RDD2022/test1_images/ {imgsz} {conf_thres} {iou_thres} {filename}




/home/testdingwenchao/ShiYu_SeaView_GRDDC2022
/home/testdingwenchao/ShiYu_SeaView_GRDDC2022/mmdetection


# Merge results

In [ ]:
#merge two results
python merge.py {file1} {file2} {output filename}